In [11]:
import numpy as np
import pandas as pd

from nltk.corpus import stopwords

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

import tensorflow as tf

In [12]:
mbti=pd.read_csv('.\data\\training\\mbti.csv')

In [13]:
mbti['type'].unique()

array(['ENFJ', 'ENFP', 'ENTJ', 'ENTP', 'ESFJ', 'ESFP', 'ESTJ', 'ESTP',
       'INFJ', 'INFP', 'INTJ', 'INTP', 'ISFJ', 'ISFP', 'ISTJ', 'ISTP'],
      dtype=object)

In [14]:
def preprocess_inputs(df):
    
    texts = df['posts'].copy()
    labels = df['type'].copy()
    
    # Process text data
    stop_words = stopwords.words('english')
    
    texts = [text.lower() for text in texts]
    texts = [text.split() for text in texts]
    texts = [[word.strip() for word in text] for text in texts]
    texts = [[word for word in text if word not in stop_words] for text in texts]
    
    vocab_length = 10000
    
    tokenizer = Tokenizer(num_words=vocab_length)
    tokenizer.fit_on_texts(texts)
    
    texts = tokenizer.texts_to_sequences(texts)
    
    max_seq_length = np.max([len(text) for text in texts])
    
    texts = pad_sequences(texts, maxlen=max_seq_length, padding='post')
    
    # Process label data
    label_values = [
        'INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP',
       'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ'
    ]
    
    label_mapping = {label: np.int(label[0] == 'E') for label in label_values}
    
    labels = labels.replace(label_mapping)
    labels = np.array(labels)
    
    return texts, labels, max_seq_length, vocab_length, label_mapping

In [15]:
texts, labels, max_seq_length, vocab_length, label_mapping = preprocess_inputs(mbti)


In [16]:
print("Text sequences:\n", texts.shape)
print("\nLabels:\n", labels.shape)
print("\nMax sequence length:\n", max_seq_length)
print("\nVocab length:\n", vocab_length)
print("\nLabel mapping:\n", label_mapping)

Text sequences:
 (8675, 860)

Labels:
 (8675,)

Max sequence length:
 860

Vocab length:
 10000

Label mapping:
 {'INFJ': 0, 'ENTP': 1, 'INTP': 0, 'INTJ': 0, 'ENTJ': 1, 'ENFJ': 1, 'INFP': 0, 'ENFP': 1, 'ISFP': 0, 'ISTP': 0, 'ISFJ': 0, 'ISTJ': 0, 'ESTP': 1, 'ESFP': 1, 'ESTJ': 1, 'ESFJ': 1}


In [20]:
import tensorflow as tf
from keras.utils import np_utils

from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split

In [21]:
texts_train, texts_test, labels_train, labels_test = train_test_split(texts, labels, test_size=0.3, random_state=1234)

In [25]:
inputs = tf.keras.Input(shape=(max_seq_length,))
embedding_dim = 512

model = Sequential()
model.add(Embedding(
    input_dim=vocab_length,
    output_dim=embedding_dim,
    input_length=max_seq_length, 
    inputs))
model.add(LSTM(128))
model.add(Dense(16, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
history = model.fit(texts_train, labels_train, epochs=5, batch_size=32)




SyntaxError: positional argument follows keyword argument (<ipython-input-25-23173eb62967>, line 9)

In [ ]:
model.load_weights('./model.h5')

In [ ]:
model.evaluate(texts_test, labels_test)